In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install onnx onnxruntime


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


# Q1

In [3]:
import onnx

model = onnx.load("hair_classifier_v1.onnx")
output_names = [node.name for node in model.graph.output]
input_names  = [node.name for node in model.graph.input]

print("Inputs:", input_names)
print("Outputs:", output_names)

Inputs: ['input']
Outputs: ['output']


In [4]:
!pip install pillow


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [6]:
import onnx
from onnx import numpy_helper

model = onnx.load("hair_classifier_v1.onnx")

print("Initializers (constants):")
for init in model.graph.initializer:
    arr = numpy_helper.to_array(init)
    print(init.name, arr.shape, arr.flatten()[:5])

Initializers (constants):
conv1.weight (32, 3, 3, 3) [ 0.1418569   0.1559236  -0.04873989  0.17149644 -0.04530386]
conv1.bias (32,) [-0.10402735  0.16268905  0.1353164   0.04503866  0.0037277 ]
fc1.weight (64, 313632) [-0.00064789 -0.0007478  -0.00045674 -0.00070455  0.00159309]
fc1.bias (64,) [ 0.00050002 -0.00078648  0.00014756 -0.0011925   0.0011093 ]
fc2.weight (1, 64) [-0.00019756  0.09418929 -0.02608154 -0.07227667 -0.05408363]
fc2.bias (1,) [0.01086449]
val_4 (1,) [313632]


In [7]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

img = download_image(url)
img = prepare_image(img, (200, 200))

import numpy as np
x = np.array(img).astype("float32")
x = x / 255.0   # this is the preprocessing

print(x[0, 0, 0])  # first pixel, R channel

0.23921569


In [8]:
import onnxruntime as ort
import numpy as np

# load session
session = ort.InferenceSession("hair_classifier_v1.onnx")

# x is your preprocessed image of shape (200, 200, 3) with x = x / 255
x_input = np.expand_dims(np.transpose(x, (2, 0, 1)), 0)  # (1, 3, 200, 200)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

pred = session.run([output_name], {input_name: x_input})[0]
print(pred)

[[1.9217669]]


In [9]:
logit = float(pred[0])
prob = 1 / (1 + np.exp(-logit))
print(prob)

0.8723353339132053


/tmp/ipykernel_2846/368752778.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  logit = float(pred[0])
